In [1]:
# Exercício de Machine Learning: 
# Criar um Modelo de ML, usando RNA Sequencial, p/ o Diagnóstico de doenças em soja.

In [31]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [33]:
# Upload dos Dados:
dataset = pd.read_csv('../dados/soybean.csv')
dataset.head()

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker


In [35]:
dataset.shape

(683, 36)

In [37]:
soybean_attributes = dataset.iloc[:,0:35].values
soybean_attributes

array([['october', 'normal', 'gt-norm', ..., 'norm', 'absent', 'norm'],
       ['august', 'normal', 'gt-norm', ..., 'norm', 'absent', 'norm'],
       ['july', 'normal', 'gt-norm', ..., 'norm', 'absent', 'norm'],
       ...,
       ['june', 'lt-normal', '?', ..., '?', '?', 'rotted'],
       ['april', 'lt-normal', '?', ..., '?', '?', 'rotted'],
       ['june', 'lt-normal', '?', ..., '?', '?', 'rotted']], dtype=object)

In [39]:
soybean_disease = dataset.iloc[:,35].values
soybean_disease

array(['diaporthe-stem-canker', 'diaporthe-stem-canker',
       'diaporthe-stem-canker', 'diaporthe-stem-canker',
       'diaporthe-stem-canker', 'diaporthe-stem-canker',
       'diaporthe-stem-canker', 'diaporthe-stem-canker',
       'diaporthe-stem-canker', 'diaporthe-stem-canker', 'charcoal-rot',
       'charcoal-rot', 'charcoal-rot', 'charcoal-rot', 'charcoal-rot',
       'charcoal-rot', 'charcoal-rot', 'charcoal-rot', 'charcoal-rot',
       'charcoal-rot', 'rhizoctonia-root-rot', 'rhizoctonia-root-rot',
       'rhizoctonia-root-rot', 'rhizoctonia-root-rot',
       'rhizoctonia-root-rot', 'rhizoctonia-root-rot',
       'rhizoctonia-root-rot', 'rhizoctonia-root-rot',
       'rhizoctonia-root-rot', 'rhizoctonia-root-rot', 'phytophthora-rot',
       'phytophthora-rot', 'phytophthora-rot', 'phytophthora-rot',
       'phytophthora-rot', 'phytophthora-rot', 'phytophthora-rot',
       'phytophthora-rot', 'phytophthora-rot', 'phytophthora-rot',
       'phytophthora-rot', 'phytophthora-rot'

In [41]:
label_encoder = LabelEncoder()
soybean_disease = label_encoder.fit_transform(soybean_disease)
for t in range (soybean_attributes.shape[1]):
    soybean_attributes[:,t] = label_encoder.fit_transform(soybean_attributes[:,t]).astype('float32')
soybean_disease

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [43]:
soybean_disease_dummy = to_categorical(soybean_disease)
soybean_disease_dummy

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
train_attributes, test_attributes, train_disease, test_disease = train_test_split(soybean_attributes, soybean_disease_dummy,
                                                                                 test_size = 0.3,
                                                                                 random_state = 0)
print(len(train_attributes), len(test_attributes), len(train_disease), len(test_disease))

478 205 478 205


In [47]:
sc = StandardScaler()
train_attributes = sc.fit_transform(train_attributes)
test_attributes = sc.transform(test_attributes)
test_attributes

array([[-0.17567151,  0.92347455, -0.45057756, ...,  0.51257859,
         0.2585467 , -0.02893995],
       [-1.21235033, -0.7613378 ,  1.99687781, ..., -2.25592498,
        -1.98845918,  1.7002223 ],
       [ 0.34266789, -0.7613378 , -1.67430524, ..., -2.25592498,
        -1.98845918,  1.7002223 ],
       ...,
       [ 1.37934671,  0.92347455, -0.45057756, ...,  0.51257859,
         0.2585467 , -0.02893995],
       [-0.17567151, -0.7613378 , -0.45057756, ...,  0.51257859,
         0.2585467 ,  1.7002223 ],
       [ 0.34266789, -0.7613378 ,  1.99687781, ..., -0.87167319,
         2.50555258, -3.48726446]])

In [49]:
RNA_model = Sequential()
RNA_model.add(Dense(units = soybean_attributes.shape[1] + 1, input_dim = soybean_attributes.shape[1]))
RNA_model.add(Dense(units = soybean_attributes.shape[1]))
RNA_model.add(Dense(units = dataset['class'].nunique(), activation = 'softmax'))

/home/guilherme-lazzarini/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
RNA_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 36)             │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 35)             │         1,295 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 19)             │           684 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,275 (12.79 KB)

 Trainable params: 3,275 (12.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
RNA_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])
RNA_model.fit(train_attributes, train_disease, epochs = 1000,
             validation_data = (test_attributes, test_disease))

Epoch 1/1000


I0000 00:00:1733173112.051340    6354 service.cc:148] XLA service 0x726260005160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733173112.051471    6354 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-12-02 17:58:32.091036: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733173112.239953    6354 cuda_dnn.cc:529] Loaded cuDNN version 90501
2024-12-02 17:58:32.847484: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.85. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


 1/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0312 - loss: 4.0111

I0000 00:00:1733173113.229339    6354 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 147ms/step - accuracy: 0.0572 - loss: 3.6964 - val_accuracy: 0.1610 - val_loss: 2.9525
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2379 - loss: 2.6824 - val_accuracy: 0.3073 - val_loss: 2.3462
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4221 - loss: 2.0692 - val_accuracy: 0.4732 - val_loss: 1.9066
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5444 - loss: 1.6718 - val_accuracy: 0.5610 - val_loss: 1.5948
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6216 - loss: 1.3499 - val_accuracy: 0.6098 - val_loss: 1.3744
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7307 - loss: 1.1042 - val_accuracy: 0.6683 - val_loss: 1.1993
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7675 - loss: 0.9559 - val_accuracy: 0.7073 - val_loss: 1.0494
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7774 - loss: 0.8491 - val_accuracy: 0.7317 

In [2]:
prevision = RNA_model.predict(train_attributes)


NameError: name 'RNA_model' is not defined